In [1]:
import unittest.mock as mock

In [2]:
mk = mock.Mock()

In [3]:
json = mk

In [4]:
display(json)

<Mock id='2228331719312'>

In [5]:
json.dumps()

<Mock name='mock.dumps()' id='2228331772504'>

In [6]:
json.loads('{"k":"v"}').get('k')

<Mock name='mock.loads().get()' id='2228331798712'>

In [7]:
json.loads.assert_called()

In [8]:
json.loads.assert_called_once()

In [9]:
json.loads.call_count

1

In [10]:
json.loads.call_args

call('{"k":"v"}')

In [11]:
json.loads.call_args_list

[call('{"k":"v"}')]

In [12]:
json.method_calls

[call.dumps(), call.loads('{"k":"v"}')]

#####  One reason to use mocks is to control your code’s behavior during tests. One way to do this is to specify a function’s return value.

In [13]:
from datetime import datetime

def is_weekday():
    today = datetime.today()
    # Python's datetime library treats Monday as 0 and Sunday as 6
    return (0 <= today.weekday() < 5)

In [14]:
assert not is_weekday()

In [15]:
import datetime
tuesday = datetime.datetime(year=2019, month=1, day=1)
saturday = datetime.datetime(year=2019, month=1, day=5)


In [16]:
datetime =mock.Mock()

In [17]:
# Mock .today() to return Tuesday
datetime.today.return_value = tuesday
# Test Tuesday is a weekday
assert is_weekday()
# Mock .today() to return Saturday
datetime.today.return_value = saturday
# Test Saturday is not a weekday
assert not is_weekday()

In [18]:
import requests
import unittest
from unittest.mock import Mock

def get_holidays():
    r = requests.get('http://localhost/api/holidays')
    if r.status_code  == 200:
        return r.json()
    return None


requests = Mock()

class TestHolidayCalendar(unittest.TestCase):
    
    def log_request(self,url):
        print(f'mocking request:{url}')
        print('Received request')
        response_mock = Mock()
        response_mock.status_code = 200
        response_mock.json.return_value  =  { '12/25':'Christmas', '8/15':'Independence Day'}
        return response_mock
    
    def test_get_holidays_logging(self):
        # Test a successful, logged request
        requests.get.side_effect = self.log_request
        self.assertEqual(get_holidays()['12/25'], 'Christmas')
        

In [19]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

.

mocking request:http://localhost/api/holidays
Received request



----------------------------------------------------------------------
Ran 1 test in 0.007s

OK


*.side_effect* can also be an iterable. The iterable must consist of return values, exceptions, or a mixture of both. The iterable will produce its next value every time you call your mocked method.

The first time you call get_holidays(), get() raises a Timeout. The second time, the method returns a valid holidays dictionary. These side effects match the order they appear in the list passed to .side_effect.

You can set .return_value and .side_effect on a Mock directly. However, because a Python mock object needs to be flexible in creating its attributes, there is a better way to configure these and other settings.

In [20]:
import unittest
from requests.exceptions import Timeout
from unittest.mock import Mock

# Mock requests to control its behavior
requests = Mock()

def get_holidays():
    r = requests.get('http://localhost/api/holidays')
    if r.status_code == 200:
        return r.json()
    return None

class TestCalendar(unittest.TestCase):
    def test_get_holidays_retry(self):
        # Create a new Mock to imitate a Response
        response_mock = Mock()
        response_mock.status_code = 200
        response_mock.json.return_value = {
            '12/25': 'Christmas',
            '7/4': 'Independence Day',
        }
        # Set the side effect of .get()
        requests.get.side_effect = [Timeout, response_mock]
        # Test that the first request raises a Timeout
        with self.assertRaises(Timeout):
            get_holidays()
        # Now retry, expecting a successful response
        assert get_holidays()['12/25'] == 'Christmas'
        # Finally, assert .get() was called twice
        assert requests.get.call_count == 2

In [21]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..

mocking request:http://localhost/api/holidays
Received request



----------------------------------------------------------------------
Ran 2 tests in 0.015s

OK


In [22]:
import unittest
from requests.exceptions import Timeout
from unittest.mock import Mock

# Mock requests to control its behavior
requests = Mock()

def get_holidays():
    r = requests.get('http://localhost/api/holidays')
    if r.status_code == 200:
        return r.json()
    return None

class TestCalendar(unittest.TestCase):
    def test_get_holidays_retry(self):
        # Create a new Mock to imitate a Response
        holidays = {'12/25': 'Christmas', '7/4': 'Independence Day'}
        response_mock = Mock(**{'json.return_value': holidays, 'status_code' : 200})
        
        # Set the side effect of .get()
        requests.get.side_effect = [Timeout, response_mock]
        # Test that the first request raises a Timeout
        with self.assertRaises(Timeout):
            get_holidays()
        # Now retry, expecting a successful response
        assert get_holidays()['12/25'] == 'Christmas'
        # Finally, assert .get() was called twice
        assert requests.get.call_count == 2

In [23]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

..

mocking request:http://localhost/api/holidays
Received request



----------------------------------------------------------------------
Ran 2 tests in 0.004s

OK


patch()
unittest.mock provides a powerful mechanism for mocking objects, called patch(), which looks up an object in a given module and replaces that object with a Mock.

Usually, you use patch() as a decorator or a context manager to provide a scope in which you will mock the target object.

patch() as a Decorator
If you want to mock an object for the duration of your entire test function, you can use patch() as a function decorator.

To see how this works, reorganize your my_calendar.py file by putting the logic and tests into separate files:

#### Let’s say you only want to mock one method of an object instead of the entire object. You can do so by using patch.object().

For example, .test_get_holidays_timeout() really only needs to mock requests.get() and set its .side_effect to Timeout:

@patch.object(requests, 'get', side_effect=requests.exceptions.Timeout)